In [94]:
import BioSimSpace as BSS
import pandas as pd
import os

In [5]:
'''
node = BSS.Gateway.Node("FEP")
node.addAuthor(name = "Ryan")
node.setLicense("GPLv3")

node.addInput("ProteinFile", BSS.Gateway.File(help = "Input the protein file."))
node.addInput("RefLigandFile", BSS.Gateway.FileSet(help = "Input the reference ligand file."))
node.addInput("LigandFiles", BSS.Gateway.FileSet(help = "Input ligand files."))

node.showControls()
'''

In [39]:
# Read the protein, reference ligand, and several test ligands via nodes
'''
protein = BSS.IO.readMolecules(node.getInput("ProteinFile"))[0]
ref_ligand = BSS.IO.readMolecules(node.getInput("RefLigandFile"))[0]
ligands = [BSS.IO.readMolecules(i)[0] for i in node.getInput("LigandFiles")]
'''

Decompressing...
Mpro-P0025_0B.pdb
Mpro-P0022_0A.pdb
Decompressing...
Mpro-P0025_0B.pdb
Mpro-P0022_0A.pdb
Decompressing...
Mpro-P0025_0B.pdb
Mpro-P0022_0A.pdb


In [92]:
# Without using nodes...
# Protein, reference ligand, and test ligand files in separate directories.

ligands_name = !ls ligands

protein = BSS.IO.readMolecules('protein/*')[0]
ref_ligand = BSS.IO.readMolecules('ref_ligand/*')[0]
ligands = {title.split('.')[0] : BSS.IO.readMolecules('ligands/' + title)[0] for title in ligands_name}

In [95]:
# Parametrise the protein and the reference ligand. (?)
protein = BSS.Parameters.ff14SB(protein).getMolecule()
ref_ligand = BSS.Parameters.gaff2(ref_ligand).getMolecule()

ParameterisationError: Parameterisation failed! Last error: 'tLEaP added missing atoms. The topology is now inconsistent with the original molecule. Please make sure that your initial molecule has a complete topology.'

In [93]:
def getFreeEnergy(title, ligand):
    # Parametrise the ligand.
    ligand = BSS.Parameters.gaff2(ligand).getMolecule()
    
    # Morph and combine systems.
    mapping = BSS.Align.matchAtoms(ligand, ref_ligand)
    ligand = BSS.Align.RMSDalign(ligand, ref_ligand, mapping)
    merged = BSS.Align.merge(ligand, ref_ligand, mapping)
    complx = merged + protein 

    # Solvate the protein ligand complex in a box of water molecules.
    complx_sol = BSS.Solvent.tip3p(molecule = complx, box = 3*[90*BSS.Units.Length.angstrom])
    # Solvate the merged ligand itself in a smaller box of water molecules.
    merged_sol = BSS.Solvent.tip3p(molecule = merged, box = 3*[30*BSS.Units.Length.angstrom])
    
    # Create the minimisation protocol.
    prot_minimisation = BSS.Protocol.Minimisation(steps = 1000)
    # Minimise both legs.
    proc_minimisation_complx = BSS.MD.run(complx_sol, prot_minimisation)
    complx_minimised = proc_minimisation_complx.getSystem(block = True)
    proc_minimisation_merged = BSS.MD.run(merged_sol, prot_minimisation)
    merged_minimised = proc_minimisation_merged.getSystem(block = True)

    # Create the equilibration protocol.
    prot_equilibration = BSS.Protocol.Equilibration(timestep = 2*BSS.Units.femtosecond,
                                                     runtime = 2*BSS.Units.nanosecond)
    # Equilibrate both legs.
    proc_equilibration_complx = BSS.MD.run(complx_minimised, prot_equilibration)
    complx_equilibrated = proc_equilibration_complx.getSystem(block = True)
    proc_equilibration_merged = BSS.MD.run(merged_minimised, prot_equilibration)
    merged_equilibrated = proc_equilibrationn_merged.getSystem(block = True)

    # Create the free energy protocols.
    prot_bound = BSS.Protocol.FreeEnergy(num_lam = 20)
    prot_free = BSS.Protocol.FreeEnergy(num_lam = 12)
    # Initialise the free energy object for each leg.
    fep_bound = BSS.FreeEnergy.Relative(complx_equilibrated,
                                        prot_bound,
                                        engine = "somd",
                                        work_dir = title + "/bound")
    fep_free = BSS.FreeEnergy.Relative(merged_equilibrated,
                                       prot_free,
                                       engine = "somd",
                                       work_dir = title + "/free")
    
    # Get the PMF and the overlap matrix.
    pmf_bound, overlap_bound = fep_bound.analyse()
    pmf_free, overlap_free = fep_free.analyse()
    
    # Compute the relative free-energy difference with PMF.
    binding_free_energy = BSS.FreeEnergy.Relative.difference(pmf_bound, pmf_free)
    
    return binding_free_energy

In [ ]:
# Run and save the result to a csv file.

df = pd.DataFrame()
for title, ligand in ligands.items():
    try:
        FreeEnergy = getFreeEnergy(title, ligand)
        df = df.append({'ligand': title, 'RelativeFE': FreeEnergy[0], 'STD': FreeEnergy[1], 'Error': '0'}, ignore_index = True)
    except:
        df = df.append({'ligand': title, 'Error': '1'}, ignore_index = True)
        
if not os.path.exists('FEP_result.csv'):
    df.to_csv('FEP_result.csv', index = False)
else:
    df.to_csv('FEP_result_1.csv', index = False)

In [ ]:
'''
node.setOutput()
node.validate()
'''